In [ ]:
# %pip install gurobipy

In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [2]:
# Resource and job sets
surplus = ['Settle', 'San Francisco']
deficit = ['Boise','Salt Lake', 'Los Angeles']
weight_lb = 1
selling_price_usd = 40


In [3]:
supply = dict({'Seattle': 30.0,
               'San Francisco': 10.0})

demand = dict({'Boise': 5.0,
               'Salt Lake': 25.0 ,
               'Los Angeles': 40.0})


In [4]:
supply

{'Seattle': 30.0, 'San Francisco': 10.0}

In [5]:
demand

{'Boise': 5.0, 'Salt Lake': 25.0, 'Los Angeles': 40.0}

In [6]:
# 1: Original Cost
# cost of shipping 1 lb package from A to B
units, cost = gp.multidict({
    ('Seattle', 'Boise'): 10.65,
    ('Seattle', 'Salt Lake'): 10.65,
    ('Seattle', 'Los Angeles'): 11.13,
    ('San Francisco', 'Boise'): 10.65,
    ('San Francisco', 'Salt Lake'):10.65,
    ('San Francisco', 'Los Angeles'): 10.65
})

In [23]:
# 2: Small Cost Change (Made LA and Salt Lake city expensive)
# # cost of shipping 1 lb package from A to B
units, cost2 = gp.multidict({
    ('Seattle', 'Boise'): 5,
    ('Seattle', 'Salt Lake'): 15,
    ('Seattle', 'Los Angeles'): 15,
    ('San Francisco', 'Boise'): 5,
    ('San Francisco', 'Salt Lake'):15,
    ('San Francisco', 'Los Angeles'): 15
})

In [ ]:
# 3: Radical Cost Change 
# cost of shipping 1 lb package from A to B
units, cost3 = gp.multidict({
    ('Seattle', 'Boise'): 5,
    ('Seattle', 'Salt Lake'): 10.65,
    ('Seattle', 'Los Angeles'): 200,
    ('San Francisco', 'Boise'): 0,
    ('San Francisco', 'Salt Lake'): 0,
    ('San Francisco', 'Los Angeles'): 100 
})

In [7]:
# revenue from shipping 1 lb jeans product from A to B 
# Assuming a constant
units, revenue = gp.multidict({
    ('Seattle', 'Boise'): 40.0,
    ('Seattle', 'Salt Lake'): 40.0,
    ('Seattle', 'Los Angeles'): 40.0,
    ('San Francisco', 'Boise'): 40.0,
    ('San Francisco', 'Salt Lake'): 40.0,
    ('San Francisco', 'Los Angeles'): 40.0
})

In [8]:
# distance dict
units, distance = gp.multidict( {
    ('Seattle', 'Boise'): 404.33,
    ('Seattle', 'Salt Lake'): 699.8,
    ('Seattle', 'Los Angeles'): 959.92,
    ('San Francisco', 'Boise'): 517.95,
    ('San Francisco', 'Salt Lake'): 599.22,
    ('San Francisco', 'Los Angeles'): 347.58})


# Decision Variables

In [24]:
model = gp.Model('BabyGurobiModel1')
transfer = model.addVars(units, name="transfer")
transfer

{('Seattle', 'Boise'): <gurobi.Var *Awaiting Model Update*>,
 ('Seattle', 'Salt Lake'): <gurobi.Var *Awaiting Model Update*>,
 ('Seattle', 'Los Angeles'): <gurobi.Var *Awaiting Model Update*>,
 ('San Francisco', 'Boise'): <gurobi.Var *Awaiting Model Update*>,
 ('San Francisco', 'Salt Lake'): <gurobi.Var *Awaiting Model Update*>,
 ('San Francisco', 'Los Angeles'): <gurobi.Var *Awaiting Model Update*>}

# Constraints

In [25]:
supply_constraints = model.addConstrs((gp.quicksum(transfer.select(city, '*')) <= supply[city] for city in supply),
                                      name="supply_const")

In [26]:
demand_constraints = model.addConstrs((gp.quicksum(transfer.select('*', city)) <= demand[city] for city in demand), 
                                      name = "demand_const")

# Objective Function

Using Cost 1

In [12]:
# Maximise profit (revenue - cost(shipping cost from s to d))
model.setObjective((transfer.prod(revenue) - transfer.prod(cost)), GRB.MAXIMIZE)
model.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 7 5800U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0x3006f87d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 4e+01]
Presolve time: 0.00s
Presolved: 5 rows, 6 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7562000e+32   1.200000e+31   1.756200e+02      0s
       3    1.1740000e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.174000000e+03


Using Cost 2

In [27]:
# Maximise profit (revenue - cost(shipping cost from s to d))
model.setObjective(transfer.prod(revenue) - transfer.prod(cost2), GRB.MAXIMIZE)
model.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 7 5800U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0x447832fa
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 4e+01]
Presolve time: 0.00s
Presolved: 5 rows, 6 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7000000e+32   1.200000e+31   1.700000e+02      0s
       5    1.0500000e+03   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.050000000e+03


Using Cost 3

In [ ]:
# Maximise profit (revenue - cost(shipping cost from s to d))
model.setObjective(transfer.prod(revenue) - transfer.prod(cost3), GRB.MAXIMIZE)
model.optimize()

In [28]:
unit_transfer = pd.DataFrame(columns=["model_id", "path_name", "surplus", "deficit", "transfer_units", "profits", "revenue","cost"])
for unit in units:
    if transfer[unit].x >= 0:
        unit_transfer = unit_transfer.append({"model_id": 2,              
                                              "path_name" : unit[0] + "_" + unit[1],
                                              "surplus": unit[0], 
                                              "deficit": unit[1], 
                                              "transfer_units": transfer[unit].x, 
                                             "cost": cost2[unit]*transfer[unit].x,
                                             "revenue": revenue[unit]*transfer[unit].x,
                                             "profits": (revenue[unit] - cost2[unit])*transfer[unit].x},
                                             ignore_index=True)  
unit_transfer.index=[''] * len(unit_transfer)
unit_transfer

C:\Users\priya\AppData\Local\Temp\ipykernel_40224\1443657919.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unit_transfer = unit_transfer.append({"model_id": 2,
C:\Users\priya\AppData\Local\Temp\ipykernel_40224\1443657919.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unit_transfer = unit_transfer.append({"model_id": 2,
C:\Users\priya\AppData\Local\Temp\ipykernel_40224\1443657919.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unit_transfer = unit_transfer.append({"model_id": 2,
C:\Users\priya\AppData\Local\Temp\ipykernel_40224\1443657919.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unit_transfer = unit_transfer.app

,model_id,path_name,surplus,deficit,transfer_units,profits,revenue,cost
,2,Seattle_Boise,Seattle,Boise,0.0,0.0,0.0,0.0
,2,Seattle_Salt Lake,Seattle,Salt Lake,25.0,625.0,1000.0,375.0
,2,Seattle_Los Angeles,Seattle,Los Angeles,5.0,125.0,200.0,75.0
,2,San Francisco_Boise,San Francisco,Boise,5.0,175.0,200.0,25.0
,2,San Francisco_Salt Lake,San Francisco,Salt Lake,0.0,0.0,0.0,0.0
,2,San Francisco_Los Angeles,San Francisco,Los Angeles,5.0,125.0,200.0,75.0


In [ ]:
# unit_transfer.to_csv("Model_2.csv")